                                                                                 ETL - REVIEWS

- Librerias a usar

In [1]:
import pandas as pd
import numpy as np
import ast
import json
from datetime import datetime

- Leemos el archivo reviwes

In [2]:
rows = []
with open('australian_user_reviews.json', encoding='utf-8') as f:
    for line in f.readlines():
        rows.append(ast.literal_eval(line))
        
df_reviews = pd.DataFrame(rows)

- Vemos la informacion del archivo

In [3]:
df_reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25799 entries, 0 to 25798
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   user_id   25799 non-null  object
 1   user_url  25799 non-null  object
 2   reviews   25799 non-null  object
dtypes: object(3)
memory usage: 604.8+ KB


In [4]:
# Crear una nueva lista para almacenar las filas expandidas
expanded_rows = []
# Iterar sobre cada fila del DataFrame
for index, row in df_reviews.iterrows():
    # Iterar sobre cada diccionario en la lista 'reviews' de la fila actual
    for review in row['reviews']:
        # Crear una nueva fila con valores de 'user_id' y 'user_url' repetidos y valores del diccionario
        new_row = {
            'user_id': row['user_id'],
            'user_url': row['user_url'],
            'item_id': review['item_id'],
            'funny': review['funny'],
            'posted': review['posted'],
            'last_edited': review['last_edited'],
            'helpful': review['helpful'],
            'recommend': review['recommend'],
            'review': review['review']
        }
        expanded_rows.append(new_row)
# Crear un nuevo DataFrame con las filas expandidas
expanded_df = pd.DataFrame(expanded_rows)

In [5]:
expanded_df

,user_id,user_url,item_id,funny,posted,last_edited,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,,"Posted November 5, 2011.",,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,,"Posted July 15, 2011.",,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,,"Posted April 21, 2011.",,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,251610,,"Posted June 24, 2014.",,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,227300,,"Posted September 8, 2013.",,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,70,,Posted July 10.,,No ratings yet,True,a must have classic from steam definitely wort...
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,362890,,Posted July 8.,,No ratings yet,True,this game is a perfect remake of the original ...
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,273110,1 person found this review funny,Posted July 3.,,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,730,,Posted July 20.,,No ratings yet,True,:D


Se hace la siguiente consulta: expanded_df[~expanded_df['posted'].str.contains(',')].shape

Tenemos '10119' valores sin el año por lo que no se tomará en cuenta las fechas que no tengan el año.

In [6]:
# Se reemplaza el nombre de la columna 'posted' por 'posted_date'
expanded_df.rename(columns={'posted': 'posted_date'}, inplace=True)
# Crear una función para convertir el formato de fecha
def transform_date_format(date_str):
    parts = date_str.split(' ')
    month_mapping = {
        'January': '01', 'February': '02', 'March': '03', 'April': '04',
        'May': '05', 'June': '06', 'July': '07', 'August': '08',
        'September': '09', 'October': '10', 'November': '11', 'December': '12'
    }
    if len(parts) == 4 and parts[0] == 'Posted' and parts[2][-1] == ',':
        year = parts[3][:-1]
        month = month_mapping.get(parts[1])
        day = parts[2][:-1]
        if month and day.isdigit() and year.isdigit():
            return f'{year}-{month}-{day.zfill(2)}'
    return None

- Aplicar la función para transformar el formato de fecha

In [7]:
expanded_df['posted_date'] = expanded_df['posted_date'].apply(transform_date_format)

In [8]:
expanded_df

,user_id,user_url,item_id,funny,posted_date,last_edited,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,,2011-11-05,,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,,2011-07-15,,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,,2011-04-21,,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,251610,,2014-06-24,,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,227300,,2013-09-08,,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,70,,None,,No ratings yet,True,a must have classic from steam definitely wort...
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,362890,,None,,No ratings yet,True,this game is a perfect remake of the original ...
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,273110,1 person found this review funny,None,,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,730,,None,,No ratings yet,True,:D


Los datos sin el año se convierten en datos nulos.
Revisamos que hay en la columa review tenemos otros idiomas en los textos por lo que para este análisis solo se
Tomará en cuenta el idioma Inglés
- Importamos las librerías necesarias para aplicar el análisis de sentimiento

In [9]:
from textblob import TextBlob
from textblob import Word
import nltk
from nltk.corpus import stopwords

- nltk.download('wordnet'): WordNet se utiliza en tareas de lematización y búsqueda de sinónimos. La lematización es el proceso de reducir una palabra a su forma base o lema. Por ejemplo, lematizar la palabra "corriendo" a "correr".

- nltk.download('stopwords'): . Las "stopwords" son palabras comunes como "a", "the", "and", "in", etc., 

In [10]:
nltk.download('wordnet')

nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Santiago\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Santiago\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

- Definir una función para normalizar el texto y eliminar stopwords

In [11]:
def preprocess_text(text, language='english'):
    words = TextBlob(text).words
    words = [Word(word).lemmatize() for word in words if word.lower() not in stopwords.words(language)]
    return ' '.join(words)

- Aplicar la función de preprocesamiento al DataFrame

In [12]:
expanded_df['normalized_review'] = expanded_df['review'].apply(preprocess_text)

- Creamos la función de analisis de sentimiento

In [13]:
def sentiment_analysis(text):
    if isinstance(text, str) and len(text) > 0:
        analysis = TextBlob(text)
        sentiment_score = analysis.sentiment.polarity
        if sentiment_score < -0.2:
            return 0  # Malo
        elif sentiment_score > 0.2:
            return 2  # Positivo
        else:
            return 1  # Neutral
    else:
        return 1  # Valor por defecto si no hay reseña

- Aplicar la función de análisis de sentimiento y crear la nueva columna

In [14]:
expanded_df['sentiment_analysis'] = expanded_df['normalized_review'].apply(sentiment_analysis)

In [15]:
expanded_df

,user_id,user_url,item_id,funny,posted_date,last_edited,helpful,recommend,review,normalized_review,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,,2011-11-05,,No ratings yet,True,Simple yet with great replayability. In my opi...,Simple yet great replayability opinion zombie ...,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,,2011-07-15,,No ratings yet,True,It's unique and worth a playthrough.,'s unique worth playthrough,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,,2011-04-21,,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,Great atmosphere gunplay bit chunky time end d...,1
3,js41637,http://steamcommunity.com/id/js41637,251610,,2014-06-24,,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,know think see title Barbie Dreamhouse Party i...,1
4,js41637,http://steamcommunity.com/id/js41637,227300,,2013-09-08,,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,simple 's actually simple truck driving Simula...,1
...,...,...,...,...,...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,70,,None,,No ratings yet,True,a must have classic from steam definitely wort...,must classic steam definitely worth buying,2
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,362890,,None,,No ratings yet,True,this game is a perfect remake of the original ...,game perfect remake original half life persona...,1
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,273110,1 person found this review funny,None,,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,much fun plaing collecting resource xD first t...,1
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,730,,None,,No ratings yet,True,:D,,1


- Reemplazar valores vacíos en la columna 'funny' y 'last_edited' por NaN

In [16]:
expanded_df['funny'] = expanded_df['funny'].replace('', pd.NA)
expanded_df['last_edited'] = expanded_df['last_edited'].replace('', pd.NA)

Debemos eliminar las columnas funnny y last_edited dado que tienen mas del 75% como nulos, e igualmente eliminamos las 
columnas review y normalized_review para quedarnos solo con sentiment_analysis.

In [17]:
expanded_df.drop(columns=['funny', 'last_edited', 'review', 'normalized_review'], inplace=True)

- Transformamos los valores booleanos de la columna recommend en binarios

In [18]:
expanded_df['recommend'] = expanded_df['recommend'].astype(int)

- Eliminamos la columna helpful dado que contiene 50% de registros sin evaluación

In [19]:
expanded_df.drop(columns=['helpful'], inplace=True)

- Transformar la columna 'posted_date' a tipo de dato fecha

In [20]:
expanded_df['posted_date'] = pd.to_datetime(expanded_df['posted_date'])

In [21]:
expanded_df

,user_id,user_url,item_id,posted_date,recommend,sentiment_analysis
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,2011-11-05,1,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,22200,2011-07-15,1,2
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,43110,2011-04-21,1,1
3,js41637,http://steamcommunity.com/id/js41637,251610,2014-06-24,1,1
4,js41637,http://steamcommunity.com/id/js41637,227300,2013-09-08,1,1
...,...,...,...,...,...,...
59300,76561198312638244,http://steamcommunity.com/profiles/76561198312...,70,NaT,1,2
59301,76561198312638244,http://steamcommunity.com/profiles/76561198312...,362890,NaT,1,1
59302,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,273110,NaT,1,1
59303,LydiaMorley,http://steamcommunity.com/id/LydiaMorley,730,NaT,1,1


                                                        Transformar el arhcivo en un CSV 

In [22]:
expanded_df.to_csv('reviews.csv', index=False)

In [23]:
df= pd.read_csv('reviews.csv')

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59305 entries, 0 to 59304
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   user_id             59305 non-null  object
 1   user_url            59305 non-null  object
 2   item_id             59305 non-null  int64 
 3   posted_date         49186 non-null  object
 4   recommend           59305 non-null  int64 
 5   sentiment_analysis  59305 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 2.7+ MB
